In [1]:
import sys
from pathlib import Path

ROOT = Path.cwd().parents[0]  # notebooks/ -> project root
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.eval import eval_split, metrics_at_threshold, pick_threshold_max_f1, pick_threshold_target_sens, find_errors_binary

print("Imported eval module OK.")


Imported eval module OK.


In [8]:
fp_paths = [test_items[i]["image"] for i in fp_f1[:10]]
fn_paths = [test_items[i]["image"] for i in fn_f1[:10]]

out_dir = r"medimg_baseline_cls\outputs\gradcam_test_errors"
records_fp = save_gradcam_batch(model, gradcam, fp_paths, out_dir + r"\FP", cfg.class_names, cfg.device, cfg.image_size)
records_fn = save_gradcam_batch(model, gradcam, fn_paths, out_dir + r"\FN", cfg.class_names, cfg.device, cfg.image_size)

print("Saved FP Grad-CAM panels:", len(records_fp))
print("Saved FN Grad-CAM panels:", len(records_fn))


NameError: name 'fp_f1' is not defined

In [6]:
from src.config import Config, seed_everything, ensure_dirs

cfg = Config(
    data_root=r"C:\Users\hyeon\Documents\miniconda_medimg_env\data\chest_xray",
    batch_size=32,
    num_workers=2,   # try 2 or 4 later
)

seed_everything(cfg.seed, deterministic=cfg.deterministic)
ensure_dirs(cfg)

print(cfg)


Config(project_name='medimg_baseline_cls', data_root='C:\\Users\\hyeon\\Documents\\miniconda_medimg_env\\data\\chest_xray', output_root='outputs', class_names=('NORMAL', 'PNEUMONIA'), pos_class_name='PNEUMONIA', image_size=(224, 224), rebuild_balanced_val=True, val_n_per_class=200, batch_size=32, num_workers=2, pin_memory=True, max_epochs=10, head_epochs=2, lr_head=0.003, lr_finetune=0.001, weight_decay=0.0001, use_weighted_sampler=True, seed=42, deterministic=False, device='cuda', save_best_by='val_ap', save_history=True, gradcam_alpha=0.35)


In [12]:
import sys
from pathlib import Path

ROOT = Path.cwd().parents[0]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.data import build_datasets, build_loaders, label_counts

DATA_ROOT = r"C:\Users\hyeon\Documents\miniconda_medimg_env\data\chest_xray"

ds = build_datasets(
    root_dir=DATA_ROOT,
    class_names=cfg.class_names,
    image_size=cfg.image_size,
    rebuild_balanced_val=True,
    val_n_per_class=200,
    seed=42,
)

print("Counts:")
print("train:", label_counts(ds["train_items"], len(cfg.class_names)))
print("val  :", label_counts(ds["val_items"], len(cfg.class_names)))
print("test :", label_counts(ds["test_items"], len(cfg.class_names)))

loaders = build_loaders(
    train_ds=ds["train_ds"],
    val_ds=ds["val_ds"],
    test_ds=ds["test_ds"],
    train_items=ds["train_items"],
    class_names=cfg.class_names,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
    pin_memory=cfg.pin_memory,
    use_weighted_sampler=True,
)

b = next(iter(loaders["train_loader"]))
print("Batch image shape:", b["image"].shape)
print("Batch label counts:", {int(k): int((b["label"].numpy()==k).sum()) for k in set(b["label"].numpy().tolist())})


Counts:
train: {0: 1141, 1: 3675}
val  : {0: 200, 1: 200}
test : {0: 234, 1: 390}
Batch image shape: torch.Size([32, 3, 224, 224])
Batch label counts: {0: 20, 1: 12}


In [14]:
import sys
import importlib
from pathlib import Path

ROOT = Path.cwd().parents[0]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import src.models as models
importlib.reload(models)

print("models.py loaded from:", models.__file__)
print("Has get_head_prefixes?", hasattr(models, "get_head_prefixes"))
print("Available:", [n for n in dir(models) if "head" in n.lower()])


models.py loaded from: C:\Users\hyeon\Documents\miniconda_medimg_env\medimg_baseline_cls\src\models.py
Has get_head_prefixes? True
Available: ['get_head_prefixes']


In [15]:
import sys
from pathlib import Path

ROOT = Path.cwd().parents[0]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.models import build_model, get_gradcam_target_layer, freeze_backbone, unfreeze_all, get_head_prefixes
from src.config import Config

cfg = Config(
    data_root=r"C:\Users\hyeon\Documents\miniconda_medimg_env\data\chest_xray",
)

arch = "resnet18"
model = build_model(arch=arch, num_classes=len(cfg.class_names), pretrained=True, device=cfg.device)
target_layer = get_gradcam_target_layer(model, arch)
print("Model:", arch, "Target layer:", target_layer)

# Freeze backbone for head-only stage
freeze_backbone(model, head_prefixes=get_head_prefixes(arch))
print("Trainable tensors:", sum(p.requires_grad for p in model.parameters()), "/", len(list(model.parameters())))


Model: resnet18 Target layer: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Trainable tensors: 2 / 62


In [ ]:
import sys
from pathlib import Path

ROOT = Path.cwd().parents[0]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.config import Config, seed_everything, ensure_dirs
from src.data import build_datasets, build_loaders
from src.models import build_model, freeze_backbone, get_head_prefixes
from src.train import run_training

cfg = Config(
    data_root=r"C:\Users\hyeon\Documents\miniconda_medimg_env\data\chest_xray",
    max_epochs=10,
    head_epochs=2,
)

seed_everything(cfg.seed, cfg.deterministic)
ensure_dirs(cfg)

# Data
ds = build_datasets(
    root_dir=cfg.data_root,
    class_names=cfg.class_names,
    image_size=cfg.image_size,
    rebuild_balanced_val=cfg.rebuild_balanced_val,
    val_n_per_class=cfg.val_n_per_class,
    seed=cfg.seed,
)
loaders = build_loaders(
    train_ds=ds["train_ds"],
    val_ds=ds["val_ds"],
    test_ds=ds["test_ds"],
    train_items=ds["train_items"],
    class_names=cfg.class_names,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
    pin_memory=cfg.pin_memory,
    use_weighted_sampler=cfg.use_weighted_sampler,
)

# Model
arch = "resnet18"
model = build_model(arch=arch, num_classes=len(cfg.class_names), pretrained=True, device=cfg.device)

# Freeze backbone for head-only stage
freeze_backbone(model, head_prefixes=get_head_prefixes(arch))

# Train
result = run_training(
    cfg=cfg,
    model=model,
    train_loader=loaders["train_loader"],
    val_loader=loaders["val_loader"],
    test_loader=loaders["test_loader"],
)

print("Run saved to:", result["run_dir"])
print("Best checkpoint:", result["best_ckpt_path"])
print("VAL summary:", result["val_summary"])
print("TEST summary:", result["test_summary"])



=== Stage A: Head-only training ===

[Epoch 1/10] (Head-only)


[Head] tr_loss=0.3069 tr_acc~=0.912 preds={0: 169, 1: 151} | val_loss=0.1963 val_acc=0.930 val_AP=0.986 | test_AP=0.955 | 248.4s

[Epoch 2/10] (Head-only)


[Head] tr_loss=0.1714 tr_acc~=0.975 preds={0: 175, 1: 145} | val_loss=0.1371 val_acc=0.950 val_AP=0.990 | test_AP=0.954 | 256.7s

=== Stage B: Full fine-tuning ===

[Epoch 3/10] (Fine-tuning)


Train (ft):  40%|██████████████████████▌                                 | 61/151 [01:28<02:05,  1.40s/it, loss=0.0624]